In [1]:
import sys;sys.path.insert(0, '..')
from src.RMSE import BatchRMSE
import torch
import os
from src.UNET import UNET
import src.core as core
import src.guassian as guassian


cwd = os.getcwd()
FCN8_model_dir = os.path.join(cwd, '..', 'models', 'UNET_FINAL_NO_AUG')

run_dir = os.path.join(FCN8_model_dir, 'run')
model_dir = os.path.join(FCN8_model_dir, 'model')


print(f"Using {core.TorchDevice} device")
torch.set_default_device(core.TorchDevice)

loaded_model = UNET()
saved_model_path = model_path = os.path.join(model_dir, 'model_20240407_120859_21')
checkpoint = torch.load(saved_model_path, map_location=core.TorchDevice)
loaded_model.load_state_dict(checkpoint)
loaded_model.eval()

rmse = BatchRMSE()

predicted_heatmaps = None
single_sample = None
for i, sample in enumerate(core.TestDataSet):
    single_sample = sample
    image, heatmaps = sample
    image, heatmaps = image.to(core.TorchDevice), heatmaps.to(core.TorchDevice)

    ## We need to unsqueeze add the batch dimension to the image [1, 3, 96, 96]
    image = image.unsqueeze(0)
    # print("image.shape", image.shape)

    predicted_heatmaps = loaded_model(image)

    pred_heatmaps = predicted_heatmaps.cpu().detach()

    ## We need to remove the extra batch dimensions from the prediction [24, 96, 96]
    pred_heatmaps = pred_heatmaps.squeeze(0)

    predicted_keypoints = guassian.heatmaps_to_keypoints_CoM(pred_heatmaps)

    ground_heatmaps = heatmaps.cpu().detach()

    gt_keypoints = guassian.heatmaps_to_keypoints_CoM(ground_heatmaps)

    rmse.add_pred_error(gt_keypoints, predicted_keypoints)


print(f"All Keypoints RMSE {rmse.get_all_keypoints_RMSE()}")



Using cpu device
All Keypoints RMSE 1686.55358191271


In [2]:
for i in range(24):
    try:
        print(f"All Keypoints RMSE {rmse.get_keypoint_RMSE(i)}")
    except:
        print(f"No RMSE for {i}th keypoint")

All Keypoints RMSE 10.215544461236945
All Keypoints RMSE 7.559499459365626
All Keypoints RMSE 22.88218063261038
All Keypoints RMSE 15.494338570159726
All Keypoints RMSE 11.142588236798032
All Keypoints RMSE 9.239798118519019
All Keypoints RMSE 13.048420504686593
All Keypoints RMSE 7.548850930006757
All Keypoints RMSE 6.968023573197425
All Keypoints RMSE 16.593993472292226
All Keypoints RMSE 35.01066580033809
All Keypoints RMSE 137.27218617702675
All Keypoints RMSE 9175.593750282786
All Keypoints RMSE 19.769291345100964
All Keypoints RMSE 10.278145059259442
All Keypoints RMSE 11.822867323839178
All Keypoints RMSE 7.665923141078147
All Keypoints RMSE 7.813152651893776
All Keypoints RMSE 15.319280214170286
All Keypoints RMSE 13.140432050853585
No RMSE for 20th keypoint
No RMSE for 21th keypoint
No RMSE for 22th keypoint
No RMSE for 23th keypoint
